# Test functionalities


In [15]:
import uproot

from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from awkward import JaggedArray
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

branches = ['nGenPart', 'GenPart_pt', 'GenPart_eta', 'GenPart_phi', 'GenPart_mass', 'GenPart_charge', 'GenPart_pdgId', 'GenPart_mpdgId', 'GenPart_vx', 'GenPart_vy', 'GenPart_vz',]
file = ['/afs/cern.ch/work/m/mabarros/public/MonteCarlo/CMSSW_10_6_12/src/show.root']

for chunk in uproot.iterate(file, 'Events', branches=branches, entrysteps=500000, namedecode='ascii'):
    GenParticle = JaggedCandidateArray.candidatesfromcounts(chunk['nGenPart'],
                                            pt=chunk['GenPart_pt'].content,
                                            eta=chunk['GenPart_eta'].content,
                                            phi=chunk['GenPart_phi'].content,
                                            mass=chunk['GenPart_mass'].content,
                                            charge=chunk['GenPart_charge'].content,
                                            pdgId=chunk['GenPart_pdgId'].content,
                                            mpdgId=chunk['GenPart_mpdgId'].content,
                                            vx=chunk['GenPart_vx'].content,
                                            vy=chunk['GenPart_vy'].content,
                                            vz=chunk['GenPart_vz'].content,
                                           )

In [16]:
kaonid = (np.absolute(GenPart.pdgId) == 321)
Kaon = GenPart[kaonid]

pionid = (np.absolute(GenPart.pdgId) == 211)
Pion = GenPart[pionid]


In [17]:
Pion_pion = Pion.distincts()

same_charge_pion_pion = (Pion_pion.i0['charge'] * Pion_pion.i1['charge'] > 0)
Pion_pion = Pion_pion[same_charge_pion_pion]

same_vtx_pion_pion = ((Pion_pion.i0['vx'] == Pion_pion.i1['vx']) & (Pion_pion.i0['vy'] == Pion_pion.i1['vy']) & (Pion_pion.i0['vz'] == Pion_pion.i1['vz']))
Pion_pion = Pion_pion[same_vtx_pion_pion]

In [18]:
Dplus_rec = Pion_pion.cross(Kaon)
opposite_charge_dplus_rec = ( Dplus_rec.i0.i0['charge'] * Dplus_rec.i0.i1['charge'] * Dplus_rec.i1['charge'] < 0)
Dplus_rec = Dplus_rec[opposite_charge_dplus_rec]

In [19]:
Dplus_mass_cut = ((Dplus_rec.mass < 1.872) & (Dplus_rec.mass > 1.866))
Dplus_rec = Dplus_rec[Dplus_mass_cut]

In [20]:
a = Pion_pion.cross(Kaon)
type(a)

awkward.array.objects.JaggedArrayMethods

In [22]:
%%timeit
a.eta.flatten()

24.9 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


KeyboardInterrupt: 